In [986]:
import glob
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf

from tensorflow.keras import layers
import time
from tensorflow import data
import pandas as pd
from tensorflow import keras
from tensorflow.keras.layers import Input, Dense,ZeroPadding2D, BatchNormalization, Activation, Layer, ReLU, LeakyReLU,Conv2D,AveragePooling2D,UpSampling2D,Reshape,Flatten
from tensorflow.keras.models import Model


In [987]:
BATCH_SIZE = 1024


In [988]:
data = np.load('finDat.npy')

In [989]:
train_dataset = tf.data.Dataset.from_tensor_slices(data).shuffle(60000).batch(1024)


In [958]:
DSHAPE = 143

In [959]:
data.shape

(1407, 143)

In [960]:
def make_generator_model():
    model = tf.keras.Sequential()
    model.add(layers.BatchNormalization())

    model.add(layers.Dense(256, activation='relu', use_bias=False, input_shape=(DSHAPE,)))


    model.add(layers.Dense(128,activation='relu'))

    model.add(layers.Dense(64,activation='relu'))
    model.add(layers.Dense(64,activation='relu'))

    model.add(layers.Dense(DSHAPE, activation='relu', use_bias=False, input_shape=(DSHAPE,)))


    

    return model

In [961]:
generator = make_generator_model()

noise = tf.random.normal([1, DSHAPE])
generated_image = generator(noise, training=False)


In [962]:
def make_discriminator_model():
    model = tf.keras.Sequential()
    model.add(layers.BatchNormalization())

    model.add(layers.Dense(256, activation='relu', use_bias=False, input_shape=(DSHAPE,)))


    model.add(layers.Dense(128,activation='relu'))
   
    model.add(layers.Dense(128,activation='relu'))

    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    return model

In [963]:
discriminator = make_discriminator_model()

In [964]:
cross_entropy = tf.keras.losses.BinaryCrossentropy()


In [965]:
def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    return 0.5*real_loss + 0.5*fake_loss

In [966]:
def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

In [967]:
generator_optimizer = tf.keras.optimizers.legacy.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.legacy.Adam(1e-4)

In [968]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)

In [969]:
EPOCHS = 125
noise_dim = DSHAPE
num_examples_to_generate = 100

# You will reuse this seed overtime (so it's easier)
# to visualize progress in the animated GIF)
seed = tf.random.normal([num_examples_to_generate, noise_dim])

In [970]:
train_dataset

<BatchDataset element_spec=TensorSpec(shape=(None, 143), dtype=tf.float32, name=None)>

In [971]:
def train_step(images, gen_losses, disc_losses):
    noise = tf.random.normal([BATCH_SIZE, noise_dim])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)

        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)

        gen_loss = generator_loss(fake_output)

        disc_loss = discriminator_loss(real_output, fake_output)
        gen_losses = gen_losses.append(gen_loss.numpy())
        disc_losses = disc_losses.append(disc_loss.numpy())
    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

In [972]:

def train(dataset, epochs, gen_losses, disc_losses, gloss, dloss):
  for epoch in range(epochs):
    start = time.time()


    for image_batch in dataset:
        train_step(image_batch, gen_losses,disc_losses)
    print("gen_loss =" + str(gen_losses[-1]))
    print("disc_loss =" + str(disc_losses[-1]))
    gloss.append(gen_losses[-1])
    dloss.append(disc_losses[-1])

    # Produce images for the GIF as you go
    x = generate_and_save_images(generator,
                             epoch + 1,
                             seed)

    # Save the model every 15 epochs
    if (epoch + 1) % 15 == 0:
      checkpoint.save(file_prefix = checkpoint_prefix)

    print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))

  # Generate after the final epoch
  saved = generate_and_save_images(generator,
                           epochs,
                           seed)
  return saved

In [973]:
def generate_and_save_images(model, epoch, test_input):
  # Notice `training` is set to False.
  # This is so all layers run in inference mode (batchnorm).
  predictions = model(test_input, training=False)
  return predictions


In [974]:
gen_losses = []
disc_losses = []
gloss = []
dloss = []

In [975]:
final = train(train_dataset,EPOCHS, gen_losses, disc_losses, gloss, dloss)


gen_loss =0.4603932
disc_loss =0.7572322
Time for epoch 1 is 0.25175023078918457 sec
gen_loss =0.47960985
disc_loss =0.7464527
Time for epoch 2 is 0.09537220001220703 sec
gen_loss =0.4985545
disc_loss =0.73916477
Time for epoch 3 is 0.08242678642272949 sec
gen_loss =0.51702696
disc_loss =0.7301507
Time for epoch 4 is 0.08409428596496582 sec
gen_loss =0.5347676
disc_loss =0.7232249
Time for epoch 5 is 0.07982897758483887 sec
gen_loss =0.55147874
disc_loss =0.7148682
Time for epoch 6 is 0.08283114433288574 sec
gen_loss =0.5672066
disc_loss =0.7102392
Time for epoch 7 is 0.08327007293701172 sec
gen_loss =0.58182186
disc_loss =0.70584536
Time for epoch 8 is 0.08201408386230469 sec
gen_loss =0.5954981
disc_loss =0.7017609
Time for epoch 9 is 0.08187508583068848 sec
gen_loss =0.6082308
disc_loss =0.696313
Time for epoch 10 is 0.08310985565185547 sec
gen_loss =0.6200365
disc_loss =0.69234025
Time for epoch 11 is 0.0823659896850586 sec
gen_loss =0.6310426
disc_loss =0.6881592
Time for epoch 12

gen_loss =2.3123574
disc_loss =0.15721124
Time for epoch 97 is 0.08373785018920898 sec
gen_loss =2.3545053
disc_loss =0.17371866
Time for epoch 98 is 0.0847921371459961 sec
gen_loss =2.3769107
disc_loss =0.16872662
Time for epoch 99 is 0.08501315116882324 sec
gen_loss =2.40072
disc_loss =0.15601021
Time for epoch 100 is 0.08310389518737793 sec
gen_loss =2.421494
disc_loss =0.15074503
Time for epoch 101 is 0.08134198188781738 sec
gen_loss =2.4627757
disc_loss =0.16116276
Time for epoch 102 is 0.08250999450683594 sec
gen_loss =2.4731116
disc_loss =0.15591973
Time for epoch 103 is 0.08353304862976074 sec
gen_loss =2.4742482
disc_loss =0.16778529
Time for epoch 104 is 0.08397626876831055 sec
gen_loss =2.4648848
disc_loss =0.14196953
Time for epoch 105 is 0.36275196075439453 sec
gen_loss =2.5779233
disc_loss =0.14938913
Time for epoch 106 is 0.08863687515258789 sec
gen_loss =2.6080146
disc_loss =0.1506187
Time for epoch 107 is 0.08391118049621582 sec
gen_loss =2.5379725
disc_loss =0.1492978

In [978]:
final.eval

<bound method _EagerTensorBase.eval of <tf.Tensor: shape=(100, 143), dtype=float32, numpy=
array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.0801862 , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.01686485, 0.02430469, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.03326439, 0.        , ..., 0.        , 0.        ,
        0.        ]], dtype=float32)>>

In [979]:
generated_images = generator(tf.random.normal([1000, noise_dim]))


In [980]:
col = np.load('columns.npy', allow_pickle=True)

In [981]:
col.shape

(143,)

In [982]:
generated = pd.DataFrame(generated_images, columns=col)

In [983]:
save = generated.to_pickle("gen_data.pkl")


In [984]:
len(gen_losses)

250

In [985]:

plt.figure(figsize=(10,8))
plt.plot(list(range(0,EPOCHS)),gloss)
plt.plot(list(range(0,EPOCHS)),dloss)
plt.title('Model Loss over Epochs')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['generator loss','discriminator loss'])
plt.savefig('Learning_curve GANdnn')
plt.show
plt.close()